Persistance : is the ability to save and restore the state of workflow over time. It helps us in the fault tolerance if at any stage LLm crash then we can resume our workflow from previous state.



To implement persistance we create checkpoints at every super step.


different thread id's are provided each time when the workflow execute to differentiate all the different time workflow executed. If we want a particular thread then we can get it into database using thread id.


In [3]:
from langgraph.graph import StateGraph,  START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [4]:
load_dotenv()

True

In [5]:
llm = ChatGroq(
    model = "llama-3.3-70b-versatile",
    temperature=0.7
)

In [6]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explaination: str

In [7]:
def generate_joke(state: JokeState):
    prompt = f"generate a joke on given topic: {state['topic']}"
    joke = llm.invoke(prompt).content
    return {"joke": joke}

In [8]:
def generate_explaination(state: JokeState):
    prompt = f"explain this joke: {state['joke']}"
    explaination = llm.invoke(prompt).content
    return {"explaination": explaination}

In [ ]:
graph = StateGraph(JokeState)

graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explaination", generate_explaination)

graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "generate_explaination")
graph.add_edge("generate_explaination", END)

checkpointer = InMemorySaver()  # this is for cache memory only.if you want to persist the data, use a database or file-based saver.

workflow = graph.compile(checkpointer=checkpointer)

In [10]:
config1 = {"configurable":{"thread_id": "1"}}
workflow.invoke({"topic":"engineer"}, config=config1)

{'topic': 'engineer',
 'joke': 'Why did the engineer cross the road?\n\nTo get to the other side... of the blueprint! (get it?)',
 'explaination': 'A clever play on words.\n\nThe joke starts with a familiar setup: "Why did the engineer cross the road?" which is a reference to the classic joke "Why did the chicken cross the road?" The expected punchline for the chicken joke is "To get to the other side."\n\nHowever, in this version, the joke takes a twist. Instead of simply saying "To get to the other side," the punchline is "To get to the other side... of the blueprint!" This is a clever wordplay because:\n\n* "The other side" is a common phrase associated with crossing a road, but in this case, it\'s given a new meaning.\n* "Blueprint" is a term commonly used in engineering and architecture to refer to a detailed technical drawing of a building, machine, or system.\n* By saying "the other side of the blueprint," the joke is making a clever connection between the physical act of crossi

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'engineer', 'joke': 'Why did the engineer cross the road?\n\nTo get to the other side... of the blueprint! (get it?)', 'explaination': 'A clever play on words.\n\nThe joke starts with a familiar setup: "Why did the engineer cross the road?" which is a reference to the classic joke "Why did the chicken cross the road?" The expected punchline for the chicken joke is "To get to the other side."\n\nHowever, in this version, the joke takes a twist. Instead of simply saying "To get to the other side," the punchline is "To get to the other side... of the blueprint!" This is a clever wordplay because:\n\n* "The other side" is a common phrase associated with crossing a road, but in this case, it\'s given a new meaning.\n* "Blueprint" is a term commonly used in engineering and architecture to refer to a detailed technical drawing of a building, machine, or system.\n* By saying "the other side of the blueprint," the joke is making a clever connection between the phy

In [13]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'engineer', 'joke': 'Why did the engineer cross the road?\n\nTo get to the other side... of the blueprint! (get it?)', 'explaination': 'A clever play on words.\n\nThe joke starts with a familiar setup: "Why did the engineer cross the road?" which is a reference to the classic joke "Why did the chicken cross the road?" The expected punchline for the chicken joke is "To get to the other side."\n\nHowever, in this version, the joke takes a twist. Instead of simply saying "To get to the other side," the punchline is "To get to the other side... of the blueprint!" This is a clever wordplay because:\n\n* "The other side" is a common phrase associated with crossing a road, but in this case, it\'s given a new meaning.\n* "Blueprint" is a term commonly used in engineering and architecture to refer to a detailed technical drawing of a building, machine, or system.\n* By saying "the other side of the blueprint," the joke is making a clever connection between the ph

#Advantages od persistance
1. short term memory
2. fault tolerance
3. human in the loop
4. time travel
